Poniżej dokonuję obliczeń współczynników R2 dla różnych modeli. Trzeba zaznaczyć że dane te dotyczą modeli uczonych z użyciem defektywnych danych, więc wyniki uczenia nie są reprezentatywne. W celu obliczenia współczynników dla każdego pliku z danymi dokonywane jest przewidywanie dla każdego wiersza w nim zawartego.
Mediana z wartości przewidzianych stanowy wynik dla danego pliku - dla jednej "warstwy".
Współczynniki R2 są liczone dla zbiorów wyników dla poszczególnych plików i zbiorów wartości parametrów T, A, B, C dla poszczególnych plików. Wynik dla C jest błędny ale nie jestem jeszcze pewien dlaczego.

In [7]:
import torch
import torch.nn as nn
import import_ipynb
import data_acquisition  
import model_creator
import joblib
import os
import pandas as pd
from scipy import stats
from sklearn.metrics import mean_squared_error
import single_file_prediction
import data_acquisition
import predictions
from scipy.stats import pearsonr



project_folder = os.getcwd()
models_folder = os.path.join(project_folder, "models", "")
scalers_folder = os.path.join(project_folder, "scalers", "")
b_feature_scaler_path = os.path.join(scalers_folder, "bScaler_featureScaler.pkl")
b_target_scaler_path = os.path.join(scalers_folder, "bScaler_targetScaler.pkl")
c_feature_scaler_path = os.path.join(scalers_folder, "cScaler_featureScaler.pkl")
c_target_scaler_path = os.path.join(scalers_folder, "cScaler_targetScaler.pkl")

In [8]:
def list_to_float(lst):
    return [float(i) for i in lst]

In [10]:
import data_acquisition
modelTT = single_file_prediction.create_and_load(models_folder ,"modelT_48_32_16_8.pth",7,1)
modelAA = single_file_prediction.create_and_load(models_folder, "modelA_48_32_16_8.pth",7,1)
modelBB = single_file_prediction.create_and_load(models_folder, "modelB_256_256_256_128_64.pth",7,1)
modelCC = single_file_prediction.create_and_load(models_folder, "modelC_64_32_32_16.pth",7,1)
modelBBstandard = single_file_prediction.create_and_load(models_folder, "modelBstandard_64_32_32_16.pth",7,1)
modelCCstandard = single_file_prediction.create_and_load(models_folder, "modelCstandard_128_64_64_32.pth",7,1)

bScaler = joblib.load(b_feature_scaler_path)
bTargetScaler = joblib.load(b_target_scaler_path)
cTargetScaler = joblib.load(c_target_scaler_path)
cScaler = joblib.load(c_feature_scaler_path)

folder_path = os.path.join(project_folder,"datasets", "new_Si_jaw_delta", "")

data = data_acquisition.get_data_chunks(["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["T", "A", "B", "C"], folder_path)
dataBStandard = data_acquisition.get_standarized_chunks(bScaler, bTargetScaler, ["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["B"], folder_path)
dataCStandard = data_acquisition.get_standarized_chunks(cScaler, cTargetScaler, ["wavelength","psi65", "del65", "psi70", "del70", "psi75", "del75"], ["C"], folder_path)


Tmeans = []
Treals = []

Ameans = []
Areals = []

Bmeans = []
Breals = []

Cmeans = []
Creals = []

BSmeans = []
BSreals = []

CSmeans = []
CSreals = []


for i in data:

    Tmeans.append(predictions.predict_file_mean(modelTT,i[0]))
    Treals.append(i[1][0][0].item())

    Ameans.append(predictions.predict_file_mean(modelAA,i[0]))
    Areals.append(i[1][0][1])

    Bmeans.append(predictions.predict_file_mean(modelBB,i[0]))
    Breals.append(i[1][0][2])

    Cmeans.append(predictions.predict_file_mean(modelCC,i[0]))
    Creals.append(i[1][0][3])

for i in dataBStandard:

    BSmeans.append(predictions.predict_file_mean(modelBBstandard,i[0]))
    BSreals.append(i[1][0][0])

for i in dataCStandard:
    CSmeans.append(predictions.predict_file_mean(modelCCstandard,i[0]))
    CSreals.append(i[1][0][0])

Tmeans = list_to_float(Tmeans)
Treals = list_to_float(Treals)
Ameans = list_to_float(Ameans)
Areals = list_to_float(Areals)
Bmeans = list_to_float(Bmeans)
Breals = list_to_float(Breals)
Cmeans = list_to_float(Cmeans)
Creals = list_to_float(Cmeans)
BSmeans = list_to_float(BSmeans)
BSreals = list_to_float(BSreals)
CSmeans = list_to_float(CSmeans)
CSreals = list_to_float(CSreals)

rT = pearsonr(Tmeans, Treals)
rA = pearsonr(Ameans, Areals)
rB = pearsonr(Bmeans, Breals)
rC = pearsonr(Cmeans, Creals)
rBstandard = pearsonr(BSmeans, BSreals)
rCstandard = pearsonr(CSmeans, CSreals)


print("R2 dla T: " + str((float(rT[0])**2)))
print("R2 dla A: " + str((float(rA[0])**2)))
print("R2 dla B: " + str((float(rB[0])**2)))
print("R2 dla C: " + str((float(rC[0])**2)))
print("R2 dla B standard: " + str((float(rBstandard[0])**2)))
print("R2 dla C standard: " + str((float(rCstandard[0])**2)))

R2 dla T: 0.9987920297578127
R2 dla A: 0.9292244992156818
R2 dla B: 0.0020717147703308886
R2 dla C: 1.0
R2 dla B standard: 0.12998004353888806
R2 dla C standard: 0.1264915584287753
